# 7.2 A box shaped barge

In [1]:
import pandas as pd
from mms185 import simpson
from mms185.sympy import lambdify,run
import matplotlib.pyplot as plt
import numpy as np
from numpy import arctan,tan, sin, cos
from sympy import symbols, Eq
import sympy as sp
import pandas as pd
from sympy import Eq,symbols
from IPython.display import display
import numpy as np
from numpy import tan

In [20]:
meta_data = {
    'T_AP':2.70,
    'T_FP':4.50,
    'L':60,
    'B':10,
    'T':3,    
}
meta_data = pd.Series(meta_data)

In [3]:
T_AP,T_FP,L,k,y,x,m = symbols("T_AP,T_FP,L,k,y,x,m")

In [4]:
eq_y = Eq(y,k*x+m)
eq_k = Eq(k,(T_FP-T_AP)/L)
eq_m = Eq(m,T_AP)

In [5]:
eqs = [
    eq_y,
    eq_k,
    eq_m,
]
for eq in eqs:
    display(eq)

solution = sp.solve(eqs, y, k, m, dict=True)[0]
eqs_solution = {key:Eq(key,value) for key,value in solution.items()}
for eq in eqs_solution.values():
    display(eq)

Eq(y, k*x + m)

Eq(k, (-T_AP + T_FP)/L)

Eq(m, T_AP)

Eq(y, (L*T_AP - T_AP*x + T_FP*x)/L)

Eq(k, (-T_AP + T_FP)/L)

Eq(m, T_AP)

In [6]:
A,M_y,x_lever = symbols(r"A,M_y,\overline{x}")
eq_A = Eq(A,sp.Integral(y,(x,0,L)))
eq_A

Eq(A, Integral(y, (x, 0, L)))

In [7]:
eq_M_y = Eq(M_y,sp.Integral(y*x,(x,0,L)))
eq_M_y

Eq(M_y, Integral(x*y, (x, 0, L)))

In [8]:
eq_x_lever = Eq(x_lever,M_y/A)
eq_x_lever

Eq(\overline{x}, M_y/A)

In [9]:
eq_A_expanded = eq_A.subs(y,eqs_solution[y].rhs).doit()
eq_A_expanded

Eq(A, L*T_AP + L*(-T_AP + T_FP)/2)

In [10]:
eq_M_y_expanded = eq_M_y.subs(y,eqs_solution[y].rhs).doit()
eq_M_y_expanded

Eq(M_y, L**2*T_AP/2 + L**2*(-T_AP + T_FP)/3)

In [11]:
eq_x_lever_expanded = sp.simplify(eq_x_lever.subs([
    (M_y,eq_M_y_expanded.rhs),
    (A,eq_A_expanded.rhs),
    ]
))
eq_x_lever_expanded

Eq(\overline{x}, L*(T_AP + 2*T_FP)/(3*(T_AP + T_FP)))

In [42]:
calculation = {}
calculation['Delta_flooded'] = 2160
calculation['Delta_intact'] = 1800
calculation['Delta_intact'] = meta_data.L*meta_data.B*meta_data['T']
calculation['Delta_compartment'] = 360
calculation['LCB_intact'] = meta_data['L']/2
calculation['LCB_flooded'] = run(eq_x_lever_expanded, meta_data)
calculation = pd.Series(calculation)

In [43]:
Delta_intact,LCB_intact,Delta_compartment,LCG_compartment, Delta_flooded,LCB_flooded = symbols("Delta_intact,LCB_intact,Delta_compartment,LCG_compartment, Delta_flooded,LCB_flooded")

In [44]:
eq_POM = Eq(Delta_intact*LCB_intact+Delta_compartment*LCG_compartment, Delta_flooded*LCB_flooded)
eq_POM

Eq(Delta_compartment*LCG_compartment + Delta_intact*LCB_intact, Delta_flooded*LCB_flooded)

In [45]:
eq_LCG_compartment = Eq(LCG_compartment, sp.solve(eq_POM, LCG_compartment)[0])
eq_LCG_compartment

Eq(LCG_compartment, (Delta_flooded*LCB_flooded - Delta_intact*LCB_intact)/Delta_compartment)

In [50]:
calculation['LCG_compartment'] = run(eq_LCG_compartment, meta_data, **calculation)

In [52]:
calculation['h'] = meta_data.T_AP + (meta_data.T_FP-meta_data.T_AP)/meta_data.L*calculation.LCG_compartment

In [64]:
calculation['V_compartment'] = calculation.Delta_compartment/0.75  # (75% permeability of the compartment)
calculation['l'] = calculation.V_compartment/(calculation.h*meta_data.B)

In [67]:
calculation['x_bulkhead_1'] = calculation.LCG_compartment-calculation.l/2
calculation['x_bulkhead_2'] = calculation.LCG_compartment+calculation.l/2

50.925925925925924

In [66]:
calculation

Delta_flooded        2160.000000
Delta_intact         1800.000000
Delta_compartment     360.000000
LCB_intact             30.000000
LCB_flooded            32.500000
LCG_compartment        45.000000
h                       4.050000
l                      11.851852
V_compartment         480.000000
dtype: float64